<a href="https://colab.research.google.com/github/7ZXU/AI/blob/main/HandsOnMachineLearning/HandsOnCh3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MNIST 숫자 데이터셋을 다양하게 학습시키기

지도학습 
- classification
- regression

##[classification] Conv net - Keras를 이용한 학습 맛보기

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
(mnist_images, mnist_labels),_=tf.keras.datasets.mnist.load_data()
###################,_왜붙이는거지?
mnist_images

#mnist_labels.shape

In [ ]:
mnist_images.shape

In [ ]:
mnist_images = tf.cast(mnist_images[..., tf.newaxis]/255, tf.float32)
#이미지를 255로 나눠주고 float32로 바꿔줌
############################255로 왜 나눠줌?
#tf.newaxis 
#차원추가
mnist_labels = tf.cast(mnist_labels, tf.int64)
#라벨을 int64로 바꿔줌
mnist_images

In [ ]:
train_images=mnist_images[:50000, :,:,:]
train_labels=mnist_labels[:50000]

test_images=mnist_images[55000:,:,:,:]
test_labels=mnist_labels[55000:]

valid_images=mnist_images[50000:55000,:,:,:]
valid_labels=mnist_labels[50000:55000]


In [ ]:
mnist_model = tf.keras.models.Sequential([
    #keras #tf의 API
    #convolution 네트워크
    #maxpolling 네트워크
    
    tf.keras.layers.Conv2D(32, [3,3],activation='relu',input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, [3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, [3,3],activation='relu'),
    tf.keras.layers.Flatten(),
    #Dense 줄여주기
    tf.keras.layers.Dense(64,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [ ]:
mnist_model.compile(
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                    optimizer='Adam', 
                    metrics=['accuracy']                    
                    )

#과열적합 과수적합

In [ ]:
mnist_model.summary()
#.summary() #모델 개요
#Total params
#예측 모델 변수는 적당해야함
#무-전기톱 #통나무-커터칼
#과열적합 #과수적합

In [ ]:
history = mnist_model.fit(
    train_images, 
    train_labels, 
    epochs = 20, 
    validation_data=(valid_images, valid_labels)
)

In [ ]:
prediction = mnist_model.predict(test_images)
#훈련된 모델을 사용하여 이미지에 대한 예측 
#prediction #모델의 신뢰도 나타냄
prediction_argmax = np.argmax(prediction, axis = 1)
#np.argmax
#axis에 해당하는 값들 중 가장 큰 값의 인덱스 반환
#axis = 1(y축)
#가장 높은 신뢰도 가진 레이블 찾기


In [ ]:
#벡터화 연산
acc_arr = (prediction_argmax == test_labels.numpy())
acc = np.sum(acc_arr)/5000*100
print("정확도 : ", acc, '%')

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

##MLP (Multi Layer Perceptron)
기하학적 정보를 담지 못한다

In [ ]:
model=tf.keras.Sequential([
                           tf.keras.layers.Flatten(input_shape=(28,28)),
                           tf.keras.layers.Dense(128, activation='relu'),
                           tf.keras.layers.Dense(10,activation='softmax')
])

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    #loss 측정 함수 
    optimizer = 'Adam', 
    metrics = ['accuracy']
    )

In [ ]:
model.summary()

In [ ]:
history = model.fit(
    train_images, train_labels, epochs=20, 
    validation_data = (valid_images, valid_labels)
                    )

In [ ]:
prediction = model.predict(test_images)
prediction_argmax = np.argmax(prediction, axis=1)

In [ ]:
acc_arr = (prediction_argmax==test_labels.numpy())
acc = np.sum(acc_arr)/5000*100 ##########################5000으로 왜 나눔?
print("정확도 : ",acc,'%')

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8,5))
plt.show()

##뭐가 더 좋은 모델인가요? CNN VS MLP 

CNN이 더 좋은 모델이다. 
지금 분류하는 데이터는 MNIST 데이터로 이미지 데이터이다.

이미지 데이터의 경우 
RGB를 생각해보면 
가까운 픽셀끼리 비슷한 값을 갖고 
먼 픽셀끼리는 그 연관성이 떨어지기 때문에 
픽셀 배열에서 오는 
기하학적인 정보가 있다.

그런데 MLP의 경우에서는 
FLATTEN METHOD를 통해서
1차원 배열로 그 기하학적 특성을 파괴하기때문에 
공간 배열에서 오는 정보를 잃는다.

따라서 공간 특성을 유지할 수 있는 
CNN이 더 좋은 모델이다.



In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version = 1)
mnist.keys()

In [ ]:
X, y = mnist["data"], mnist["target"]

print(X.shape)
print(y.shape)

In [ ]:
type(y[0]) #숫자가 아닌 문자열

In [ ]:
y=y.astype(np.uint8) #숫자로 변환

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
#train, test data 불러오기

##**이진 분류기** 
True of False

In [ ]:
y_train_5=(y_train == 5)
t_test_5=(y_test == 5)
#5일때만 1값 가짐

In [ ]:
from sklearn.linear_model import SGDClassifier 
#SGDClassifier
#Support Vector machine과 비슷
#경사하강법 -> 모든 데이터 사용해서 w,b 구함
#확률적 경사하강법 -> 일부 데이터만 뽑아서 w, b 구함 
#SVM을 SGD로 최적화한 것
from sklearn.pipeline import make_pipeline
#pipeline
#버튼 누르면 이하 기능이 실행됨

sgd_clf = make_pipeline( #파이프라인 감싸주는 습관 들이기
    SGDClassifier(random_state=42)
)
sgd_clf.fit(X_train, y_train_5) #학습

In [ ]:
from sklearn.model_selection import cross_val_score 
#교차검증이용
#데이터 세트를 a, b, c로 나눔
#b,c 이용 ->a 테스트 
#데이터 세트 분할하는 수만큼 성능 나옴
cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring = 'accuracy')

#from~import~ #강경한 의사표현식 
#from <module> import <method>
#module #.py로 된 파이썬 라이브러리 집합


In [ ]:
import sklearn.model_selection as sms
sms.cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')
#위의 from~import~와 동일한 의미

##**Never5Classifier**
모든 이미지를 5가 아니라고 하는 더미 분류기

In [ ]:
#DuckTyping
#method를 통해서 type 결정
#다른 class더라도 동일 method이면 동일 type으로 간주
class Parrot:
    def fly(self):
        print("Parrot flying")

class Airplane:
    def fly(self):
        print("Airplane flying")

class Whale:
    def swim(self):
        print("Whale swimming")

def lift_off(entity):
    entity.fly()

parrot = Parrot()
airplane = Airplane()
whale = Whale()

lift_off(parrot) # prints `Parrot flying`
lift_off(airplane) # prints `Airplane flying`
lift_off(whale) # Throws the error `'Whale' object has no attribute 'fly'`

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
#BaseEstimator
#.fit() #transform 감 잡기
#.transform() #transform 실행

#TransformerMixin 
#fit_transform = fit+transform
#fit_transform 자동상속

class Never5Classifier(BaseEstimator, TransformerMixin): #상속
    def fit(self, X, y=None): 
        #y는 부모클래스의 self 메서드에서 정의 #따라서 y=None
        return self
    def transform(self, X, y=None):
        return self
    def predict(self, X): #모두 0, False 예측 만들어냄
        return np.zeros((len(X),1), dtype=bool)

cross_val_score(Never5Classifier(), X_train, y_train_5, cv=3, scoring='accuracy')


##**오차행렬-이진분류기**

무엇이 오차였는지 확인하기 위해서 오차 행렬 직접 확인해보기

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5,cv=3)
confusion_matrix(y_train_5, y_train_pred)
#confusion_matrix
#([[1 0],  1 : 5 아님 -> 5 아님 
#  [0 1]]) 1 : 5 맞음 -> 5 맞음



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_train_5, y_train_pred)
recall = recall_score(y_train_5, y_train_pred)
f1 = f1_score(y_train_5, y_train_pred)
print('정밀도:',precision,'\n재현율:',recall,'\nF1:',f1)


**의료 데이터에서는 정밀도가 중요한가 재현율이 중요한가**

- 정밀도 : true라고 예측한 값 중 실제 true인 비율

- 재현율 : 실제 true인 값을 true로 예측하는 비율

true를 최대한 빠짐없이 예측하는 것이 중요하기에 정밀도 보다는 재현율이 중요하다

##**정밀도 재현율 트레이드오프 : 이진분류기**

In [ ]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method='decision_function')

In [ ]:
from sklearn.metrics import precision_recall_curve
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
plt.plot(thresholds, precisions[:-1], 'b--', label='정밀도')
plt.plot(thresholds, recalls[:-1], 'g--',label='재현율')
plt.ylim([0,1])
plt.xlim([-100000, 50000])
plt.show()

In [ ]:
thresholds.shape

##**다중분류기 : 핵심**


In [ ]:
from sklearn.svm import SVC
svm_clf = SVC()
svm_clf.fit(X_train, y_train)
#데이터 전처리 및 결과해석이 더 중요

In [ ]:
def disp(prediction, ans):
    if prediction == ans : 
        print("정답")
    else:
        print("오답")

for i in range(10):
    disp(svm_clf.predict([X_test[i]]),y_test[i])

In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring='accuracy')

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
cross_val_score(rf_clf, X_train, y_train, cv=3, scoring = 'accuracy')

##**데이터 전처리의 중요성**

In [ ]:
from sklearn.preprocessing import StandardScaler
X_train_scaled=StandardScaler().fit_transform(X_train.astype(np.float64))

In [ ]:
cross_val_score(sgd_clf, X_train_scaled,y_train, cv=3, scoring='accuracy')

In [ ]:
cross_val_score(rf_clf, X_train_scaled, y_train, cv=3,scoring = 'accuracy')

##**에러분석**


In [ ]:
y_train_pred = cross_val_predict(sgd_clf, X_train_scaled, y_train, cv=3) #예측 만들기
conf_mx=confusion_matrix(y_train,t_train_pred) 
plt.matshow(conf_mx, cmap=plt.cm.gray) #이미지 표현
plt.show()

In [ ]:
row_sums=conf_mx.sum(axis=1, keepdims=True)
#오차 행렬 값을 클래스의 이미지 개수로 나눔
#에러 비율 비교 
norm_conf_mx = conf_mx / row_sums

In [ ]:
np.fill_diagonal(norm_conf_mx, 0) #주대각선 0 으로 채움
plt.matshow(norm_conf_mx, cmap=plt.cm.gray) #이미지 표현
plt.show()

#행 #실제 클래스 
#열 #예측 클래스 